some helper functions

In [189]:
import pandas as pd
import numpy as np

def safe_div_s(a, b, b_min = 0.01):
    if (b > 0 or b < 0):
        return a / b
    else:
        return a / b_min
def safe_div(df, dividend_name, divisor_name, b_min = 0.01): #a safe division of two columns
    try:
        df[divisor_name]
    except:
        print ('Warning: A column name with the name must be specified =' + divisor_name)
        return None
    try:
        df[dividend_name]
    except:
        print ('Warning: A column name with the name must be specified =' + dividend_name)
        return None
    df.loc[df[divisor_name] == 0, divisor_name] = b_min
    df[dividend_name[:7]+'/'+divisor_name[:7]] = df[dividend_name]/df[divisor_name]
    return df
def median_calculated(df, filter_column, target_column): #it calculates a list of median values filterd by filter_column
    try:
        df[filter_column]
    except:
        print ('Warning: A column name with the name must be specified =' + filter_column)
        return None
    try:
        df[target_column]
    except:
        print ('Warning: A column name with the name must be specified =' + target_column)
        return None
    my_list = df[filter_column].unique()
    #print(my_list)
    #print(df.loc[df[filter_column] == my_list[0]])
    result = []
    for el in my_list:
        #print(df.loc[df[filter_column] == el])
        #print(df.loc[df[filter_column] == el][target_column])
        result.append(df.loc[df[filter_column] == el][target_column].median())
    return result
# it calculates dividend/divisor, if divisor =0, it repaces it on median calculated with all rows in range by filter_columns
#a = [[5, 1.2, 4.2], [1, 0, 0.03], [8, 5, 1], [8,2,4], [16,4,4], [32,9,4], [32,0,4], [-9,0,2]]
#df = pd.DataFrame(a, columns=['dividend', 'diviser', 'filter'])
#print(df)
#print(median_safe_div(df, 'dividend', 'diviser', 'filter'))
#   dividend  diviser  filter
#0         5      1.2    4.20
#1         1      0.0    0.03
#2         8      5.0    1.00
#3         8      2.0    4.00
#4        16      4.0    4.00
#5        32      9.0    4.00
#6        32      0.0    4.00
#7        -9      0.0    2.00
#   dividend  diviser  filter  dividend/diviser
#0         5      1.2    4.20          4.166667
#1         1      0.0    0.03          4.000000
#2         8      5.0    1.00          1.600000
#3         8      2.0    4.00          4.000000
#4        16      4.0    4.00          4.000000
#5        32      9.0    4.00          3.555556
#6        32      0.0    4.00          4.000000
#7        -9      0.0    2.00          4.000000



def div_median(df, dividend_name, divisor_name, filter_column):
    try:
        df[divisor_name]
    except:
        print ('Warning: A column name with the name must be specified =' + divisor_name)
        return None
    try:
        df[dividend_name]
    except:
        print ('Warning: A column name with the name must be specified =' + dividend_name)
        return None
    try:
        df[filter_column]
    except:
        print ('Warning: A column name with the name must be specified =' + filter_column)
        return None
    my_list = df[filter_column].unique()
    #calculate basic
    new_name = dividend_name[:8]+'/'+divisor_name[:8]
    df[new_name] = df[dividend_name]/df[divisor_name]
    median =df[new_name].median()
    median_list = median_calculated(df, filter_column, new_name)
    j=0
    #replacing infinity on median of every range (by filter_column)
    for el in median_list:
        if (el == np.inf or el == -np.inf):
            el = df[new_name].median()
            median_list[j] = df[new_name].median()
        j+=1    

    i=0
    #if range = 1 row and it had 0 in diviser --> replace result column value on median of entire column
    for el in my_list:
        df.loc[(df[filter_column] == el)&((df[new_name]== np.inf) | (df[new_name]== -np.inf)), new_name] = median_list[i]
        i+=1
    #df[new_name].replace(to_replace=np.inf, value=median, inplace=True)
    #df[new_name].replace(to_replace=-np.inf, value=median, inplace=True)
    return (df)


main_file_path = 'train.csv'
data = pd.read_csv(main_file_path)
pd.set_option('display.max_rows', 100)
#print(median_calculated(data, 'Neighborhood', 'LotArea'))
#print(median_calculated(df, 'one', 'two'))
data = div_median(data, 'SalePrice', 'LotArea', 'Neighborhood')
data_g_p = data['SalePric/LotArea']
#print(data_g_p)
#print(data_g_p)
#print(data)
#print(data.describe())
#print(data.columns)
unique_data = data['Neighborhood'].unique()
#print(unique_data)
neighborhood = pd.get_dummies(data['Neighborhood'], columns=['Neighborhood'])
#print(neighborhood)
price_columns = ['SalePric/LotArea']

data_sparce = pd.concat([neighborhood, data_g_p], axis=1)
#print(result)
#price_lot_data = safe_div(data['SalePrice'], data['LotArea'], 1)
#print(price_lot_data)

In [190]:
from sklearn.model_selection import train_test_split
y = data_sparce['SalePric/LotArea']
dummy_cols = list(set(data_sparce.columns) - set(price_columns)) #only dummy cols
dummy_data = data_sparce[dummy_cols] 
x= dummy_data
train_x, val_x, train_y, val_y = train_test_split(x, y,random_state = 100)

In [196]:
#print(train_x.describe())
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Define model
model = LinearRegression()

# Fit model
model.fit(train_x, train_y)
predicted_soil_prices=model.predict(val_x) 
actual_soil_prices=val_y
mean_absolute_error(actual_soil_prices, predicted_soil_prices)
data_comaparasion = pd.DataFrame(np.concatenate([actual_soil_prices,  predicted_soil_prices], axis=0))
print(data_comaparasion)

             0
0    13.388889
1    16.874195
2    22.349206
3    23.093564
4    17.307692
5    10.232773
6    18.616807
7    16.892531
8    14.930556
9    15.843676
10   22.953846
11   24.625268
12   13.624238
13    5.698346
14   11.280788
15    4.907940
16   21.183432
17   14.303616
18   11.286320
19   16.366612
20   13.355531
21   17.620524
22   11.201054
23   10.398123
24   21.824287
25   20.981087
26    7.860000
27   12.150599
28   25.696186
29   12.250000
30   17.561983
31   67.271589
32   22.780147
33   14.491525
34   14.610390
35    7.304348
36   11.104257
37   14.299065
38   17.298821
39   11.966253
40   26.660027
41   24.852071
42   16.287879
43    4.705882
44   19.390244
45    6.396624
46   14.977235
47   20.633329
48   15.118110
49   10.588235
..         ...
680  15.136719
681  15.011719
682  16.972656
683  21.281250
684  16.453125
685  18.761719
686  13.652344
687  16.589844
688  15.011719
689  36.800781
690  25.703125
691  22.152344
692  21.281250
693  16.453125
694  57.77